In [83]:
import pandas as pd
import json

In [8]:
rappi = pd.read_csv('../scraper/data/rappi_revolucion258.csv')
eats = pd.read_csv('../scraper/data/ue_revolucion258.csv')

In [130]:
rappi.head(3)

,id,name,lat,lng,cuisine,platform
0,taqueria-orinoco,Taquería Orinoco Roma Norte CDMX,19.417796,-99.164973,"Mexicana, Tacos",rappi
1,koku,Koku Condesa,19.410262,-99.169336,"Sushi, Asiática",rappi
2,pizza-felix,Pizza Felix,19.418673,-99.157614,"Pizza, Italiana",rappi


In [223]:
RES = 11

In [224]:
def df_map(df_restos, res):
    df = df_restos[['platform', 'lat', 'lng']].reset_index(drop=True).copy()
    df['hex_id'] = df.apply(lambda row: h3.geo_to_h3(row['lat'], row['lng'], res=res), axis=1)
    
    polygons = []
    for h in df.hex_id:
        polygons.append({
            "type" : "Polygon"
            , "coordinates": [h3.h3_to_geo_boundary(h3_address=h, geo_json=True)]
        })
    df['geometry'] = pd.Series(polygons)
    df['res'] = res
    return(df)

In [225]:
from h3 import h3
from geojson.feature import *
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster

In [226]:
map_example = Map(
        location= [19.3987204, -99.184695]
        , zoom_start=16
        , tiles="cartodbpositron"
    )

In [227]:
cloud_kitchen_hex = h3.geo_to_h3(19.3987204, -99.184695, res=RES)
ck_feature = Feature(
    geometry = {
        "type" : "Polygon"
        , "coordinates": [h3.h3_to_geo_boundary(h3_address=cloud_kitchen_hex, geo_json=True)]
    }
    , id = cloud_kitchen_hex
    , properties = {"resolution": int(RES)}
)

ck_geojson = json.dumps(FeatureCollection([ck_feature]))

In [228]:
list_features = []
for i, row in df_map(eats, RES).iterrows():
    feature = Feature(
        geometry = row["geometry"]
        , id=row["hex_id"]
        , properties = {"resolution": int(row["res"])}
    )
    list_features.append(feature)

eats_geojson = json.dumps(FeatureCollection(list_features))

list_features = []
for i, row in df_map(rappi, RES).iterrows():
    feature = Feature(
        geometry = row["geometry"]
        , id=row["hex_id"]
        , properties = {"resolution": int(row["res"])}
    )
    list_features.append(feature)


rappi_geojson = json.dumps(FeatureCollection(list_features))

In [229]:
GeoJson(
    ck_geojson
    , style_function = lambda feature: {
        'fillColor': "blue"
        , 'weight': 2
        , 'fillOpacity': 1
    }
).add_to(map_example)

GeoJson(
    eats_geojson
    , style_function = lambda feature: {
        'fillColor': "green"
        , 'color': "green"
        , 'weight': 2
        , 'fillOpacity': 0.4
    }
    , name = "Example" 
).add_to(map_example)

GeoJson(
    rappi_geojson
    , style_function = lambda feature: {
        'fillColor': "orange"
        , 'color': "orange"
        , 'weight': 2
        , 'fillOpacity': 0.4
    }
    , name = "Example" 
).add_to(map_example)

map_example